In [38]:
import pandas as pd

College Stats from Sports References

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--user-data-dir=C:/Users/d_lasc01/AppData/Local/Google/Chrome/User Data/Profile 1')
chrome_options.add_argument("--disable-extensions")

In [40]:
import re
driver = webdriver.Chrome(options=chrome_options)
pos_ranking_url = 'https://www.sports-reference.com/cfb/schools/'
driver.get(pos_ranking_url)

t = driver.find_elements_by_css_selector("#schools > tbody > tr > td[data-stat='school_name'] > a")

c = driver.find_elements_by_css_selector("#schools > tbody > tr > td[data-stat='year_max']")

schools = []
for i in range(len(t)):
    if int(c[i].text) >= 2017:
        schools.append(re.findall('/schools/(.*)/', t[i].get_attribute("href"))[0])
    else: continue

driver.close()

In [41]:
""" year=tuple(range(2010,2022))
col_stats = dict.fromkeys(schools)

for school in schools:
    col_stats[school] = dict.fromkeys(year) """

In [328]:
""" tables = ['passing', 'rushing_and_receiving', 'defense_and_fumbles', 'kicking_and_punting', 'scoring']

for school in schools:
    for yr in year:
        col_stats[school][yr] = dict.fromkeys(tables) """

pure selenium solution slower --> hidden if code needed

In [ ]:
""" from tqdm import tqdm

driver = webdriver.Chrome(options=chrome_options)

column_n = [['Player','Games', 'Completions', 'Attempts', 'Percentage', 'Yards', 'Y/A', 'AY/A', 'TD', 'INT', 'Rate'],
            ['Player', 'Games', 'Attempts', 'R_Yards', 'R_AVG', 'R_TD', 'Catches', 'C_Yards', 'C_AVG', 'C_TD', 'S_Plays', 'S_YDS', 'S_AVG' ,'S_TD'],
            ['Player', 'Games', 'Solo', 'Assist', 'Tot', 'Loss', 'Sack', 'INT', 'INT_Y', 'INT_AVG_Y', 'INT_TD', 'PD', 'Fum_Rec', 'Fum_Y', 'Fum_TD', 'Fum_Forc'],
            ['Player', 'Games', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%','PTS', 'Punts', 'YDS', 'AVG'],
            ['Player', 'Games', 'R_TD', 'C_TD', 'INT_TD', 'FR_TD', 'PR_TD', 'KR_TD', 'Other', 'Total', 'XPM', 'FGM', '2PM', 'Safety', 'Points']]

ids = ['#passing', '#rushing_and_receiving', '#defense_and_fumbles', '#kicking_and_punting', '#scoring']

for i in tqdm(range(len(schools)), position=1):
    for j in tqdm(range(len(year)), position=0):

        data_link = 'https://www.sports-reference.com/cfb/schools/{school}/{year}.html'.format(school = schools[i], year = year[j])
        driver.get(data_link)

        
        for k in range(len(ids)):
            table_id = driver.find_element_by_css_selector(ids[k])

            rows = [[column_n[k]]]

            for x in range(len(table_id.find_elements_by_css_selector("tbody > tr"))):
                row = []

                for y in range(len(table_id.find_elements_by_css_selector("tbody>tr[data-row='0']>td"))):
                    #if table_id.find_elements_by_css_selector("tbody>tr[data-row='{no}']>td".format(no=x)) == []:
                    #    continue
                    #else: row.append(table_id.find_elements_by_css_selector("tbody>tr[data-row='{no}']>td".format(no=x))[y].text)
                    
                    try: row.append(table_id.find_elements_by_css_selector("tbody>tr[data-row='{no}']>td".format(no=x))[y].text)
                    except: continue
                rows.append(row)

            col_stats[schools[i]][year[j]][ids[k][1:]] = pd.DataFrame(rows[1:] , columns = rows[0]).dropna().reset_index(drop=True)
        
driver.close()

%store col_stats
 """

In [ ]:
""" from tqdm import tqdm
from bs4 import BeautifulSoup

driver = webdriver.Chrome(options=chrome_options)

column_n = [['Player','Games', 'Completions', 'Attempts', 'Percentage', 'Yards', 'Y/A', 'AY/A', 'TD', 'INT', 'Rate'],
            ['Player', 'Games', 'Attempts', 'R_Yards', 'R_AVG', 'R_TD', 'Catches', 'C_Yards', 'C_AVG', 'C_TD', 'S_Plays', 'S_YDS', 'S_AVG' ,'S_TD'],
            ['Player', 'Games', 'Solo', 'Assist', 'Tot', 'Loss', 'Sack', 'INT', 'INT_Y', 'INT_AVG_Y', 'INT_TD', 'PD', 'Fum_Rec', 'Fum_Y', 'Fum_TD', 'Fum_Forc'],
            ['Player', 'Games', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%','PTS', 'Punts', 'YDS', 'AVG'],
            ['Player', 'Games', 'R_TD', 'C_TD', 'INT_TD', 'FR_TD', 'PR_TD', 'KR_TD', 'Other', 'Total', 'XPM', 'FGM', '2PM', 'Safety', 'Points']]

ids = ['#passing', '#rushing_and_receiving', '#defense_and_fumbles', '#kicking_and_punting', '#scoring']

for i in tqdm(range(len(schools))):
    for j in range(len(year)):

        data_link = 'https://www.sports-reference.com/cfb/schools/{school}/{year}.html'.format(school = schools[i], year = year[j])
        driver.get(data_link)
        soup = driver.page_source
        soup = BeautifulSoup(soup, 'html.parser')

        for k in range(len(ids)):

            rows = [[column_n[k]]]

            try:
                table = soup.select(ids[k])[0]
                table_rows = table.find_all('tr')
            
                row = []
                for tr in table_rows:
                    td = tr.find_all('td')
                    row = [tr.text.strip() for tr in td]
                    if row:
                        rows.append(row)
            except: continue

            col_stats[schools[i]][year[j]][ids[k][1:]] = pd.DataFrame(rows[1:] , columns = rows[0]).dropna().reset_index(drop=True)
        
driver.close()

%store col_stats """

In [52]:
from tqdm import tqdm
from bs4 import BeautifulSoup

driver = webdriver.Chrome(options=chrome_options)

year=tuple(range(2010,2022))

column_n = [['Year','College','Player', 'Games', 'Completions', 'Attempts', 'Percentage', 'Yards', 'Y/A', 'AY/A', 'TD', 'INT', 'Rate'],
            ['Year','College','Player', 'Games', 'Attempts', 'R_Yards', 'R_AVG', 'R_TD', 'Catches', 'C_Yards', 'C_AVG', 'C_TD', 'S_Plays', 'S_YDS', 'S_AVG' ,'S_TD'],
            ['Year','College','Player', 'Games', 'Solo', 'Assist', 'Tot', 'Loss', 'Sack', 'INT', 'INT_Y', 'INT_AVG_Y', 'INT_TD', 'PD', 'Fum_Rec', 'Fum_Y', 'Fum_TD', 'Fum_Forc'],
            ['Year','College','Player', 'Games', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%','PTS', 'Punts', 'YDS', 'AVG'],
            ['Year','College','Player', 'Games', 'R_TD', 'C_TD', 'INT_TD', 'FR_TD', 'PR_TD', 'KR_TD', 'Other', 'Total', 'XPM', 'FGM', '2PM', 'Safety', 'Points']]

ids = ['#passing', '#rushing_and_receiving', '#defense_and_fumbles', '#kicking_and_punting', '#scoring']

passing = [column_n[0]] ; rush_rec = [column_n[1]] ; def_fum = [column_n[2]]; kick = [column_n[3]]; scoring = [column_n[4]]

for i in tqdm(range(len(schools))):
    for j in range(len(year)):

        data_link = 'https://www.sports-reference.com/cfb/schools/{school}/{year}.html'.format(school = schools[i], year = year[j])
        driver.get(data_link)
        soup = driver.page_source
        soup = BeautifulSoup(soup, 'html.parser')

        for k in range(len(ids)):

            rows = []

            try:
                table = soup.select(ids[k])[0]
                table_rows = table.find_all('tr')
            
                row = []
                for tr in table_rows:
                    td = tr.find_all('td')
                    row = [tr.text.strip() for tr in td]
                    if row:
                        row.insert(0, schools[i])
                        row.insert(0, year[j])
                        rows.append(row)
            except: continue

            if k == 0:
                passing.extend(rows)
            elif k == 1:
                rush_rec.extend(rows)
            elif k == 2:
                def_fum.extend(rows)
            elif k == 3:
                kick.extend(rows)
            else : scoring.extend(rows)                 
        
driver.close()

passing_df = pd.DataFrame(passing[1:] , columns = passing[0]).dropna().reset_index(drop=True)
rush_rec_df = pd.DataFrame(rush_rec[1:] , columns = rush_rec[0]).dropna().reset_index(drop=True)
def_fum_df = pd.DataFrame(def_fum[1:] , columns = def_fum[0]).dropna().reset_index(drop=True)
kick_df = pd.DataFrame(kick[1:] , columns = kick[0]).dropna().reset_index(drop=True)
scoring_df = pd.DataFrame(scoring[1:] , columns = scoring[0]).dropna().reset_index(drop=True)

passing_df.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\passing_df.csv', index=False)
rush_rec_df.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\rush_rec_df.csv', index=False)
def_fum_df.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\def_fum_df.csv', index=False)
kick_df.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\kick_df.csv', index=False)
scoring_df.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\scoring_df.csv', index=False)

100%|██████████| 132/132 [1:09:49<00:00, 31.74s/it]


Add Players where school not in database

### Consensus Big Board Data revealing communities/Scouts "value" of the Player ###
--> value does not inclue Team needs --> A team might pass on the best player in the draft (which is a DE, because they need a QB)

--> Adjustment for team needs will be done later

In [ ]:
import requests
from requests_html import HTMLSession

session = HTMLSession()
bigboard_url = ['https://www.nflmockdraftdatabase.com/big-boards/2019/consensus-big-board-2016',
                'https://www.nflmockdraftdatabase.com/big-boards/2019/consensus-big-board-2017','https://www.nflmockdraftdatabase.com/big-boards/2019/consensus-big-board-2018',
                'https://www.nflmockdraftdatabase.com/big-boards/2019/consensus-big-board-2019','https://www.nflmockdraftdatabase.com/big-boards/2020/consensus-big-board-2020',
                'https://www.nflmockdraftdatabase.com/big-boards/2021/consensus-big-board-2021', 'https://www.nflmockdraftdatabase.com/big-boards/2022/consensus-big-board-2022']

In [ ]:
con_big_boards = []

from itertools import chain
for i in range(len(bigboard_url)):
    res = session.get(bigboard_url[i])
    name_selector = res.html.find('.player-name-smaller')

    bb = [name_selector[i].element.text for i in range(len(name_selector))]
    
    year = [2016,2017,2018,2019,2020,2021,2022]
    bb = [(*el,year[i]) for el in list(enumerate(bb, start=1))]

    con_big_boards.append(bb)

con_big_boards = list(chain(*con_big_boards))

In [ ]:
con_big_boards = pd.DataFrame(con_big_boards, columns=['Player_Rank', 'Player', 'Year'])

In [ ]:
# Change tyler johnson to tyler johnson 2020
con_big_boards.Player[(con_big_boards.Player == "Tyler Johnson") & (con_big_boards.Year == 2020)] = "Tyler Johnson 2020"
con_big_boards.Player[(con_big_boards.Player == "Jonah Williams") & (con_big_boards.Year == 2020)] = "Jonah Williams 2020"


In [ ]:
con_big_boards.to_excel(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\big_boards2.xlsx')

In [ ]:
con_big_boards = pd.read_excel(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\big_boards2.xlsx')

### Team needs to adjust Pick number relative to overall value of the player ###

In [ ]:
import requests
from requests_html import HTMLSession

session = HTMLSession()
team_needs_url = [  'https://www.nflmockdraftdatabase.com/team-needs-2016',
                    'https://www.nflmockdraftdatabase.com/team-needs-2017','https://www.nflmockdraftdatabase.com/team-needs-2018',
                    'https://www.nflmockdraftdatabase.com/team-needs-2019','https://www.nflmockdraftdatabase.com/team-needs-2020',
                    'https://www.nflmockdraftdatabase.com/team-needs-2021', 'https://www.nflmockdraftdatabase.com/team-needs-2022']

In [ ]:
team_needs = []

from itertools import chain
for i in range(len(team_needs_url)):
    res = session.get(team_needs_url[i])
    team_selector = res.html.find('.medium-pick-number')
    needs_selector1 = res.html.find('.player-name-smaller')
    needs_selector2 = res.html.find('.college-details')

    team = [team_selector[i].element.text for i in range(len(team_selector))]
    needs1 = [[el.strip() for el in needs_selector1[i].element.text.split(',')] for i in range(len(needs_selector1))]
    needs2 = [[el.strip() for el in needs_selector2[i].element.text.split(',')] for i in range(len(needs_selector2))]

    needs = [list(chain(*list(el))) for el in zip(needs1,needs2)]

    year = [2016,2017,2018,2019,2020,2021,2022]
    tn = [(*el,year[i]) for el in list(zip(team,needs))]

    team_needs.append(tn)

team_needs = list(chain(*team_needs))

In [ ]:
team_needs = pd.DataFrame(team_needs, columns=['Team','Position','Year'])

Add team abbreviations:

In [ ]:
session = HTMLSession()
abbrev_url = 'https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Football_League/National_Football_League_team_abbreviations'

res = session.get(abbrev_url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'html.parser')

table=soup.find('table',{'class':"wikitable"})
table = pd.read_html(str(table))
table = pd.DataFrame(table[0])
table.columns = table.iloc[0]
table = table.drop(table.index[0]).reset_index(drop=True)

#table.to_excel(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\team_abbreviations.xlsx')


In [ ]:
table = pd.read_excel(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\team_abbreviations.xlsx')

In [ ]:
team_Column = [table.Franchise[table['Commonly Used Abbreviations'] == el].values[0]  for el in team_needs.Team ]
team_needs['Franchise'] = team_Column

team_needs.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\team_needs2.csv', index=False)

In [49]:
from ast import literal_eval

team_needs = pd.read_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\team_needs2.csv', converters={'Position' : literal_eval})

Contract/Salary Data from Spotrac

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [19]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--user-data-dir=C:/Users/d_lasc01/AppData/Local/Google/Chrome/User Data/Profile 1')
chrome_options.add_argument("--disable-extensions")


In [4]:
import pandas as pd
draft_data = pd.read_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\Combined_draft_data.csv')

In [96]:
draft_data['Team_Abbrv'] = [ team_needs.Team.drop_duplicates()[team_needs.Franchise.drop_duplicates() == el].values[0] for el in draft_data.NFL_team]

In [225]:
position_names = dict([('QB', 'Quarterback'), ('RB','Running Back'),('FB','Fullback'), ('WR','Wide Receiver'), ('TE','Tight End'),('OL','Offensive Lineman'),('C',	'Center'),('OG', 'Offensive Guard'),('LG','Left Guard'),
                  ('RG', 'Right Guard'),('OT', 'Offensive Tackle'),('LT', 'Left Tackle'),('RT','Right Tackle'),('K','Kicker'),('KR','Kick Returner'),('DL', 'Defensive Lineman'),('DE', 'Defensive End'),
                  ('DT', 'Defensive Tackle'),('NT', 'Nose Tackle'),('LB', 'Linebacker'),('ILB', 'Inside Linebacker'),('OLB', 'Outside Linebacker'),('MLB', 'Middle Linebacker'),('DB', 'Defensive Back'),
                  ('CB'	,'Cornerback'),('FS', 'Free Safety'),('SS', 'Strong Safety'),('S','Safety'), ('P','Punter'), ('LS','Long Snapper')])


In [ ]:
players = tuple(zip(draft_data.Player, [position_names[draft_data['Pos.'][i]]  for i in range(len(draft_data))]))

Adjust names in draft data

In [120]:
import re
from tqdm import tqdm

contracts = []
salary_df = [['Player','Contract_Terms','Signing_Bonus','Average_Salary','GTD_Sign','Total_GTD','Free_Agent']]
driver = webdriver.Chrome(options=chrome_options)

for i in tqdm(range(len(draft_data))):
    try:
        driver.get('https://www.spotrac.com/')


        e = driver.find_elements_by_css_selector('#search-form input')[0]
        e.send_keys(draft_data.Player[i])
        e.send_keys(Keys.RETURN)

        wait = WebDriverWait(driver, 10)
        wait.until(EC.url_changes('https://www.spotrac.com/'))

        url=driver.current_url
            
        if ('result' in url) & (len(driver.find_elements_by_css_selector('div.teamitem')) == 0):
            e = driver.find_elements_by_css_selector('#search-form input')[0]
            e.send_keys(draft_data.Player[i].split()[1])
            e.send_keys(Keys.RETURN)

            url="https://www.spotrac.com/search/results/{search_word}/".format(search_word=draft_data.Player[i].split()[1])
            if EC.url_changes(url):
                url = driver.current_url
                pass
            else: 
                continue
            
        if 'result' in url:

            search_teams = []
            for j in range(len(driver.find_elements_by_css_selector('#main img'))):
                search_teams.append(re.findall('/thumb/(.*).png', driver.find_elements_by_css_selector('#main img')[j].get_attribute('src'))[0])

            driver.execute_script("window.open('https://www.pro-football-reference.com/');")
            driver.switch_to.window(driver.window_handles[1])

            d = driver.find_element_by_css_selector("form[name='f_big'] input[class='ac-input completely']")
            
            d.send_keys(draft_data.Player[i])
            d.send_keys(Keys.RETURN)
            
            url=driver.current_url

            try:
                if 'search' in url:

                    controlls = driver.find_elements_by_css_selector('#players > div > div.search-item-name')
                    controlls = [controlls[i].text.split() for i in range(len(controlls))]
                    
                    match = [(el[0:2] == draft_data.Player[i].split()) & (int(el[3][1:5]) == draft_data.Year[i]) for el in controlls ].index(True) + 2

                    driver.find_element_by_css_selector('#players > div:nth-child({no}) > div.search-item-name > a'.format(no = match)).click()

                    try:                
                        if 'traded' not in driver.find_element_by_css_selector("#div_transactions > ul:nth-child(1) > li:nth-child(1)").text:
                            cur_team = driver.find_elements_by_css_selector("#div_transactions > ul:nth-child(1) > li:nth-child(1) > a[href]")                     
                            if len(cur_team) == 3:
                                cur_team = cur_team[1].text
                            else:  cur_team = cur_team[0].text
                        else: cur_team = driver.find_element_by_css_selector("#div_transactions > ul:nth-child(1) > li:nth-child(1) > a:nth-child(4)").text
                    
                    except: cur_team = driver.find_elements_by_css_selector("#snap_counts td a")[-1].get_attribute('title')
                    
                    cur_team = team_needs.Team[team_needs.Franchise == cur_team].iloc[0]
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                else: 
                    try:                
                        if 'traded' not in driver.find_element_by_css_selector("#div_transactions > ul:nth-child(1) > li:nth-child(1)").text:
                            cur_team = driver.find_elements_by_css_selector("#div_transactions > ul:nth-child(1) > li:nth-child(1) > a[href]")                     
                            if len(cur_team) == 3:
                                cur_team = cur_team[1].text
                            else:  cur_team = cur_team[0].text
                        else: cur_team = driver.find_element_by_css_selector("#div_transactions > ul:nth-child(1) > li:nth-child(1) > a:nth-child(4)").text
                    
                    except: cur_team = driver.find_elements_by_css_selector("#snap_counts td a")[-1].get_attribute('title')
                    cur_team = team_needs.Team[team_needs.Franchise == cur_team].iloc[0]
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except: 
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue

            idx = [re.sub(r'\d+$', '', el) for el in search_teams].index(cur_team.lower())
            
            driver.find_elements_by_css_selector('.team-name')[idx].click()
            

            contracts = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "playerValue", " " ))]')

            composite_list = [contracts[x:x+6] for x in range(0, len(contracts),6)]

            for list in composite_list:
                row = []
                row.append(draft_data.Player[i])
                
                for el in list:
                    row.append(el.text)
                salary_df.append(row)

        else:

            contracts = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "playerValue", " " ))]')

            composite_list = [contracts[x:x+6] for x in range(0, len(contracts),6)]

            for list in composite_list:
                row = []
                row.append(draft_data.Player[i])
                
                for el in list:
                    row.append(el.text)
                salary_df.append(row)

    except: continue

driver.close()


100%|██████████| 2558/2558 [7:18:14<00:00, 10.28s/it]   


In [25]:
%store -r salary_df

In [70]:
player_salary = pd.DataFrame(salary_df[1:], columns=salary_df[0])

In [3]:
#player_salary.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\player_salary.csv', index=False)
player_salary = pd.read_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\player_salary.csv')

In [87]:
missing = draft_data[[el not in list(player_salary.Player) for el in draft_data.Player]]

In [96]:
import re
from tqdm import tqdm

contracts = []
salary_df2 = [['Player','Contract_Terms','Signing_Bonus','Average_Salary','GTD_Sign','Total_GTD','Free_Agent']]
driver = webdriver.Chrome(options=chrome_options)

for i in missing.index:
    
    try:
        driver.get('https://www.spotrac.com/')


        e = driver.find_elements_by_css_selector('#search-form input')[0]
        e.send_keys(missing.Player[i])
        e.send_keys(Keys.RETURN)

        wait = WebDriverWait(driver, 10)
        wait.until(EC.url_changes('https://www.spotrac.com/'))

        url=driver.current_url
            
                   
        if 'result' in url:

            search_teams = []
            #for j in range(len(driver.find_elements_by_css_selector('#main img'))):
            #    search_teams.append(re.findall('/thumb/(.*).png', driver.find_elements_by_css_selector('#main img')[j].get_attribute('src'))[0])

            for j in range(len(driver.find_elements_by_css_selector('div.teamitem'))):

                driver.find_elements_by_css_selector('.team-name')[j].click()
                
                logo = []
                for k in range(len(driver.find_elements_by_css_selector('#contracts > table > tbody > tr > td > div > h2'))):
                    logo.append(re.sub(r'[0-9]+', '', re.findall('/thumb/(.*).png', driver.find_elements_by_css_selector('#contracts > table > tbody > tr > td > div > h2 > span.contract-type-logo > img')[k].get_attribute('src'))[0]))
            
                if team_needs.Franchise[team_needs.Team == logo[-1].upper()].unique()[0] == missing.NFL_team[i]:

                    contracts = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "playerValue", " " ))]')

                    composite_list = [contracts[x:x+6] for x in range(0, len(contracts),6)]

                    for lst in composite_list:
                        row = []
                        row.append(draft_data.Player[i])
                        
                        for el in lst:
                            row.append(el.text)
                        salary_df2.append(row)

                else: driver.back()

    except: continue

driver.close()


In [223]:
salary_df.extend(salary_df3)

In [224]:
player_salary = pd.DataFrame(salary_df[1:], columns=salary_df[0])

In [124]:
missing = draft_data[[el not in list(player_salary.Player) for el in draft_data.Player]]

In [226]:
player_salary.to_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\player_salary.csv', index=False)
#player_salary = pd.read_csv(r'C:\Users\d_lasc01\OneDrive - Universität Münster\6. Promotion\COVID and Sports\player_salary.csv')

In [225]:
%store salary_df3

Stored 'salary_df3' (list)


In [123]:
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.spotrac.com/')

In [125]:
salary_df3 = []

In [222]:
contracts = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "playerValue", " " ))]')

composite_list = [contracts[x:x+6] for x in range(0, len(contracts),6)]

for lst in composite_list:
    row = []
    row.append(missing.Player[2181])
    
    for el in lst:
        row.append(el.text)
    salary_df3.append(row)

Scraping draftscout data --> no longer needed

In [ ]:
""" import selenium
import bs4
import requests
from selenium import webdriver

position = [ 'QB', 'RB', 'FB', 'TE', 'WR', 'C', 'OT', 'OG', 'K', 'DE', 'DT', 'ILB', 'OLB', 'CB', 'FS', 'SS', 'P' ]
year = [2019,2020]
#pos_ranking_url = 'https://draftscout.com/players{yr}.php?GenPos={pos}&DraftYear={yr}&sortorder=TSXPos&order=ASC'.format(pos = position[i], yr = year[j])

pos_ranking = pd.DataFrame([],columns=["Rank","Tier","Player Name", 'College', 'Proj High','Projected','Proj Low', 'Height', 'Weight', '40 Time Range', 'Hometown, State', 'High School','Position','Year'])

for i in range(len(position)):
    for j in range(len(year)):
        driver = webdriver.Chrome()
        pos_ranking_url = 'https://draftscout.com/players{yr}.php?GenPos={pos}&DraftYear={yr}&sortorder=TSXPos&order=ASC'.format(pos = position[i], yr = year[j])
      
        driver.get(pos_ranking_url)

        # Obtain the number of rows in body
        rows = len(driver.find_elements_by_xpath("/html/body/font/b/div/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr")) -1
        cols=len(driver.find_elements_by_xpath("/html/body/font/b/div/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td")) -1
        elements = driver.find_elements_by_xpath("/html/body/font/b/div/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td")[1:]

        df=[]
        for r in range(0,rows):
            row = []
            for c in range(0, int(cols/rows)):
                row.append(elements[c+list(range(cols+1))[0::int(cols/rows)][r]].text)
                df.append(row)

        

        df = pd.DataFrame(df,columns=["Rank","Tier","Player Name", 'College', 'Proj High','Projected','Proj Low', 'Height', 'Weight', '40 Time Range', 'Hometown, State', 'High School'])
        df = df.drop_duplicates()

        df['Position'] = position[i]
        df['Year'] = year[j]

        pos_ranking = pos_ranking.append(df)

        driver.close()

 """